__Probabilistic Programming. Wasowski. Pardo. IT University of Copenhagen__

This file contains the list of exercises for the week, as well as any related code.

These exercises are based on Chapter 3, McElreath. Solve: 
* 3E1, 3E2, 3E3, 3E4, 3E5, 3E6, 3E7
* 3M1, 3M2, 3M3, 3M4, 3M5, 3M6
* 3H1, 3H2, 3H3, 3H4, 3H5

Do not waste time and skip the exercises that are too easy (that you immediately see how to solve).


In [1]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import scipy.stats as stats
import pandas as pd
import xarray as xr
import graphviz

RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)
hdi_fill_args = { 'color': 'black', 'alpha': 0.1 }

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


The easy exercises are using this code (a Python version of R3.27 on p. 68)

In [2]:
def posterior_grid_approx(grid_points=5, success=6, tosses=9):
    # define grid
    p_grid = np.linspace(0, 1, grid_points)

    # define prior (the value 1 is arbitrary, any positive constant would do)
    # prior = np.repeat(1, grid_points)  # uniform
    prior = (p_grid >= 0.5).astype(int)  # truncated
    # prior = np.exp(- 5 * abs(p_grid - 0.5))  # double exp

    # compute likelihood at each point in the grid
    likelihood = stats.binom.pmf(success, tosses, p_grid)

    # compute product of likelihood and prior (pointwise multiplication, vectorization of *)
    unstd_posterior = likelihood * prior

    # standardize the posterior, so it sums to 1 (vectorization of division by a scalar)
    posterior = unstd_posterior / unstd_posterior.sum()
    return xr.DataArray(posterior, dims="p", coords={"p": p_grid})

This code gives you a set of samples to experiment with:

In [3]:
posterior = posterior_grid_approx(grid_points=100, success=6, tosses=9)
np.random.seed(100)
samples = np.random.choice(posterior.coords['p'].data, p=posterior.data, size=1000, replace=True)

array([0.68686869, 0.60606061, 0.65656566, 0.7979798 , 0.50505051,
       0.55555556, 0.72727273, 0.78787879, 0.55555556, 0.6969697 ,
       0.81818182, 0.58585859, 0.57575758, 0.54545455, 0.58585859,
       0.88888889, 0.77777778, 0.57575758, 0.78787879, 0.60606061,
       0.65656566, 0.84848485, 0.78787879, 0.62626263, 0.57575758,
       0.63636364, 0.50505051, 0.5959596 , 0.77777778, 0.50505051,
       0.70707071, 0.70707071, 0.54545455, 0.63636364, 0.51515152,
       0.81818182, 0.88888889, 0.52525253, 0.81818182, 0.6969697 ,
       0.75757576, 0.71717172, 0.6969697 , 0.50505051, 0.58585859,
       0.68686869, 0.76767677, 0.5959596 , 0.60606061, 0.7979798 ,
       0.88888889, 0.81818182, 0.63636364, 0.70707071, 0.63636364,
       0.62626263, 0.57575758, 0.5959596 , 0.52525253, 0.67676768,
       0.63636364, 0.70707071, 0.71717172, 0.56565657, 0.84848485,
       0.85858586, 0.70707071, 0.64646465, 0.63636364, 0.58585859,
       0.60606061, 0.5959596 , 0.57575758, 0.87878788, 0.86868

## Hard

Data for Hard exercises

In [ ]:
birth1 = np.array([1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
                   1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1])
birth2 = np.array([0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0,
                   1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                   1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1,
                   0, 0, 0, 1, 1, 1, 0, 0, 0, 0])